# Augmenting Xray micro-CT data with MICP data for high resolution pore-microstructural and flow modelling of carbonate rocks.
### Extracting pore size distribution from xray data.

Lead Invsestigator: Olubukola Ishola (olubukola.ishola@okstate.edu)\
Co-Investigator: Javier Vilcaez\
Associated Paper: Augmenting Xray micro-CT data with MICP data for high resolution pore-microstructural and flow modelling of carbonate rocks.\
DOI:

The notebook demonstrates the workflow for obtaining pore size distribution from binarized micro-CT images. The general principle involves identifying each pore, estimating the volume of each pore, and ultimately determining the radius of a sphere with an equivalent volume of each pore. There is also an option to analyse the sample to see if it is representative.

### Install dependencies?

In [1]:
# !conda install -c conda-forge porespy
# !conda install -c conda-forge imageio

### Importing Packages

In [2]:
import numpy as np
import porespy as ps
import imageio as io
import os
import pandas as pd

C:\Users\geobu\anaconda3\envs\pytorch\lib\site-packages\openpnm\algorithms\_invasion_percolation.py:358: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _find_trapped_pores(inv_seq, indices, indptr, outlets):  # pragma: no cover


### Helper Functions

In [3]:
def xray_to_psd(filename,
                resolution,
                sub_volume=[100],
                file_path=False,
                save_path=False,
                return_psd=False,
                save_psd=True,
                save_rev=True):
    """
    Converts binary segmented X-ray images of rocks to pore size distribution. Pores are represented by white pixels (255), while the background is black (0). Note that "size" here refers to radius. There is also an option to analyse the sample to see if it is representative.
    
    Parameters
    ----------
    filename : str
        Name of the file including the extension.
    resolution : float
        Resolution of the image in microns.
    subvolume : list, optional
        A list specifying the percentage of the length scale to process. Must be between 0 and 100. Please note that this refers to the percentage of the length of one axis, not the percentage of the total volume. The default value is 100.
    file_path : str, optional
        Directory of the folder where the input file is located. The default value is the working directory.
    save_path : str, optional
        Directory of the folder where the output file will be saved. The default value is the working directory.
    return_psd : boolean, optional
        Indicates whether the pore size distribution should be returned. The default value is False.
    save_psd : str, optional
        Indicates whether the pore size distribution should be saved. The default value is True.
    save_rev : str, optional
        Indicates whether data on porosity against volume for REV analysis should be saved. The default value is True.

    Returns
    -------
    Pore size distribution : dataframe, optional
    """


    if file_path:
        file = os.path.join(file_path, filename)
    else:
        file = filename

    image = io.volread(file)
    image = np.invert(image)
    image //= 255
    len_ = min(image.shape)
    image_volume = image.shape[0] * image.shape[1] * image.shape[2]
    sample_porosity = np.sum(image) / image_volume

    volume_rev = []
    volume_rev.append(image_volume * ((resolution / 1000)**3))
    porosity_rev = []
    porosity_rev.append(sample_porosity)
    print(
        f"Porosity of full image is {round(sample_porosity*100, 2)}% and the corresponding volume is {round(volume_rev[0], 2)} mm3."
    )

    for sub in sub_volume:
        sub /= 100
        sub *= len_
        sub = int(sub)
        sub_image = image[:sub, :sub, :sub]

        snow = ps.filters.snow_partitioning(im=sub_image)
        regions = snow.regions * snow.im
        props = ps.metrics.regionprops_3D(regions)
        df = ps.metrics.props_to_DataFrame(props)['volume']

        sub_porosity = df.sum() / (sub**3)
        sub_image_volume = (sub * (resolution / 1000))**3
        volume_rev.append(sub_image_volume)
        porosity_rev.append(sub_porosity)
        print(
            f"Porosity of subvolume is {round(sub_porosity*100, 2)}% and the corresponding subvolume is {round(sub_image_volume, 2)} mm3."
        )

        excel_name = str(
            sub_image_volume)[:5] + '_mm3_' + filename[:-4] + '.xlsx'

        if save_path:
            excel_name = os.path.join(save_path, excel_name)

        df = pixel_volume_to_pore_radius(df, resolution=resolution)

        if save_psd:
            custom_headers = ['Pore radius (microns)']
            df.to_excel(excel_name, header=custom_headers, index=False)

        if return_psd:
            return df

    if save_rev:
        rev_df = pd.DataFrame({
            'volume (mm3)': volume_rev,
            'porosity (%)': porosity_rev,
        })

        rev_name = filename[:-4] + '.xlsx'

        if save_path:
            rev_name = os.path.join(save_path, rev_name)

        rev_df.to_excel(rev_name, index=False)

In [4]:
def pixel_volume_to_pore_radius(pixel_volume, resolution):
    """
    Converts pixel volume to the radius of an equivalent sphere.      

    Parameters
    ----------
    pixel_volume : float
        Volume reported by porespy in pixels.
    resolution : float
        Resolution of image in microns.

    Returns
    -------
    Pore radius : float
        pore radiusof equivalent sphere.
    """

    pixel_pore_radius = ((3 / 4) * (1 / 3.14) * pixel_volume)**(1 / 3)
    pore_radius = pixel_pore_radius * resolution

    return pore_radius

### Inputs

In [5]:
filename = 'SampleA.tif'
file_path = r'.'
resolution = 7.5
sub_volume = [5]

### Run Program

In [6]:
xray_to_psd(filename, resolution, sub_volume, file_path=file_path)

Porosity of full image is 11.97% and the corresponding volume is 421.87 mm3.


0it [00:00, ?it/s]

Porosity of subvolume is 17.5% and the corresponding subvolume is 0.05 mm3.
